In [ ]:
#importing libraries and packages
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import keras.backend as K
from PIL import Image
import glob
import sys
import PIL
import os.path
import cv2
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.applications import  Xception,VGG16,InceptionResNetV2
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras import datasets, layers, models
import sklearn
import PIL
from skimage import transform, color
from matplotlib import image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import PIL.Image
from PIL import UnidentifiedImageError
from keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, Reshape, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from os import listdir
from os.path import isfile, join
from keras.layers import Dropout

In [ ]:
#Load in the data 
infected_path = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized"
uninfected_path = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/Uninfected"
infected_files = glob.glob(infected_path+'/*.png')
uninfected_files = glob.glob(uninfected_path+'/*.png')

print(len(infected_files) + len(uninfected_files))

np.random.seed(42)

In [ ]:
#adding the images together and making a dataframe
files_df = pd.DataFrame({
    'filename': infected_files + uninfected_files,
    'label': ['malaria'] * len(infected_files) + ['healthy'] * len(uninfected_files)
}).sample(frac=1, random_state=42).reset_index(drop=True)

files_df.head()

In [ ]:
sample_images_positive = []
i=0
for filename in listdir ("/kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized/"):
    if i < 10:
        img = plt.imread("/kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized/" + filename)
        sample_images_positive.append(img)
        i+=1

In [ ]:
sample_images_negative = []
i=0
for filename in listdir("/kaggle/input/cell-images-for-detecting-malaria/cell_images/Uninfected/"):
    if i < 10:
        img = plt.imread('/kaggle/input/cell-images-for-detecting-malaria/cell_images/Uninfected/' + filename)
        sample_images_negative.append(img)
        i+=1

In [ ]:
plt.figure()
fig, axes = plt.subplots(1, 4, figsize=(15,15))
ax1 = axes[0]
ax1.imshow(sample_images_positive[0], cmap='Greys_r')
ax1.set_title('infected')
ax1.axis('off')
ax2 = axes[1]
ax2.imshow(sample_images_positive[5], cmap='Greys_r')
ax2.set_title('infected')
ax2.axis('off')
ax3 = axes[2]
ax3.imshow(sample_images_positive[9], cmap='Greys_r')
ax3.set_title('infected')
ax3.axis('off')
ax4 = axes[3]
ax4.imshow(sample_images_negative[0], cmap='Greys_r')
ax4.set_title('uninfected')
ax4.axis('off');

In [ ]:
image_grayscale = color.rgb2gray(sample_images_positive[5])
plt.imshow(image_grayscale, cmap='Greys_r')

In [ ]:
def padding(image, size):
    desired_size = size
    old_size = image.size
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im = image.resize(new_size, Image.ANTIALIAS)
    new_im = Image.new('RGB',(desired_size, desired_size))
    new_im.paste(image, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))
    return new_im

In [ ]:
def brighten(image):
    max_ = np.max(image)
    image[image==0.0] = max_
    return image

In [ ]:
img = sample_images_positive[5]
img = np.array(img)
img = color.rgb2gray(img)
img = brighten(img)
plt.imshow(img, cmap='Greys_r');

In [ ]:
# Check if the images are the same size
# get image
img1 = Image.open(infected_files[0])
# show the image
plt.imshow(img1)
plt.axis('off')
plt.show();
# get width and height
width1 = img1.width
height1 = img1.height
# display width and height
print("The height of image 1 is: ", height1)
print("The width of image 1 is: ", width1)
# get image
img2 = Image.open(uninfected_files[0])
# show the image
plt.imshow(img2)
plt.axis('off')
plt.show();
# get width and height
width2 = img2.width
height2 = img2.height
# display width and height
print("The height of image 2 is: ", height2)
print("The width of image 2 is: ", width2)
del img1
del img2
del infected_files
del uninfected_files

In [ ]:
# #resizing all the images
resized_df = []
labels_names = []
for file in range(0, len(files_df)):
    img_array = cv2.imread(files_df.iloc[file,0])
    img = Image.fromarray(img_array, 'RGB')
    img_resized = img.resize((75,75))
    resized_df.append(np.array(img_resized))
    labels_names.append(files_df.iloc[file,1])

del files_df

# recode labels to numbers instead of names
labels_recoded = []
for i, j in enumerate(labels_names):
    if j == "malaria":
        labels_recoded.append(1)
    else:
        labels_recoded.append(0)
data = np.array(resized_df)
labels = np.array(labels_recoded)

del resized_df
del labels_names
del labels_recoded

data = data.astype(np.float32)
labels = labels.astype(np.int32)
data = data/255

In [ ]:
# #Check if resizing worked
print("There are", data.shape[0], "pictures.") #All picteres are there!
print("All picture have a width of", data.shape[1], "and a length of", data.shape[2], "pixels.")
print("And all picture have", data.shape[3], "color channels.")


In [ ]:
# split data set into train, validation and test datset using 60:10:30 split
train_data, test_data, train_labels, test_labels = train_test_split(data,
                                                                    labels, 
                                                                    test_size = 0.3,
                                                                    random_state = 42)
train_data, val_data, train_labels, val_labels = train_test_split(train_data,
                                                                  train_labels,
                                                                  test_size = 0.1,
                                                                  random_state = 42)

In [ ]:
print("Train files:", len(train_data),"\nVal files:", len(val_data),"\nTest files:", len(test_data))
print('Train labels:', len(train_labels), '\nVal labels:', len(val_labels), '\nTest labels:', len(test_labels))

In [ ]:
def freezing_layers(model_name):
#     for layer in model_name.layers[0:6]:
      layer.trainable = False  

In [ ]:
Parasitized= os.listdir('/kaggle/input/cell-images-for-detecting-malaria/cell_images/Parasitized/')
Uninfected = os.listdir('/kaggle/input/cell-images-for-detecting-malaria/cell_images/Uninfected/')
image_path =  '../input/cell-images-for-detecting-malaria/cell_images/cell_images/'

In [ ]:
plt.figure(figsize=(20,15))
plt.suptitle("Parasitized Images", fontsize=20)
path =image_path+'/'+'Parasitized/'
counter =0
for i,img in enumerate(os.listdir(path))  :
        plt.subplot(5,7,i+1)
        full_image= plt.imread(path+img)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.xlabel('Parasitized')
        plt.imshow(full_image, cmap=plt.cm.binary) 
        if i == 34:
            break

In [ ]:
plt.figure(figsize=(20,15))
plt.suptitle("Uninfected Images", fontsize=20)
path =image_path+'/'+'Uninfected/'
counter =0
for i,img in enumerate(os.listdir(path))  :
        plt.subplot(5,7,i+1)
        full_image= plt.imread(path+img)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.xlabel('Uninfected')
        plt.imshow(full_image, cmap=plt.cm.binary) 
        if i == 34:
            break

In [ ]:
#basic CNN model
model_cnn = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(75,75,
3)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 #tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 #tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 #tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu'),
 tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
model_cnn.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
                loss='binary_crossentropy',
                metrics=['accuracy'])
model_cnn.summary()

In [ ]:
plt.figure(figsize=(5, 10))
plot_model(model_cnn, to_file="model.png")

In [ ]:
history = model_cnn.fit(train_data,
                    train_labels, 
                    validation_data = (val_data,val_labels),
                    batch_size = 64,
                    epochs=10,
                    verbose=1)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Basic CNN Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

max_epoch = len(history.history['accuracy'])+1
epoch_list = list(range(1,max_epoch))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(1, max_epoch, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")
ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(1, max_epoch, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

history_frame = pd.DataFrame(history.history)
print("Maximum validation accuracy: {}".format(history_frame['accuracy'].max()))

In [ ]:
def show_model_result(x_test, y_test, model):

#     y_test_c = np.argmax(y_test, axis=1)
    target_names = ["Uninfected", "Parasitized"]

    y_pred = np.argmax(model.predict(x_test),axis=1)
    print('Confusion Matrix')
    cm = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(cm, target_names, normalize=False, title='Confusion Matrix')

    print('Classification Report')
    print(classification_report(y_test, y_pred, target_names=target_names))
    
    return cm

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
cm = show_model_result(test_data, test_labels, model_cnn)

In [ ]:
with open('cnn_basic.npy', 'wb') as f:
    np.save(f, history.history['accuracy'])
    np.save(f, history.history['val_accuracy'])
    np.save(f, history.history['loss'])
    np.save(f, history.history['val_loss'])
    np.save(f, range(len(history.history['val_loss'])))
    np.save(f, cm)

In [ ]:
#CNN model with L1 Regularizer
from tensorflow.keras import regularizers
model2 = tf.keras.models.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(75, 75,
3)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 #tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 #tf.keras.layers.BatchNormalization(),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu' , kernel_regularizer = regularizers.l1(0.002)),
 tf.keras.layers.Dense(1, activation='sigmoid') # 2 in case of softmax
])

In [ ]:
model2.summary()

In [ ]:
#compiling the model
model2.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=['accuracy'])

In [ ]:
history = model2.fit(train_data,
                    train_labels, 
                    validation_data = (val_data,val_labels),
                    batch_size = 64,
                    epochs=10,
                    verbose=1)

In [ ]:
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.figure()
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
cm = show_model_result(test_data, test_labels, model2)

In [ ]:
with open('cnn_plus_L1.npy', 'wb') as f:
    np.save(f, history.history['accuracy'])
    np.save(f, history.history['val_accuracy'])
    np.save(f, history.history['loss'])
    np.save(f, history.history['val_loss'])
    np.save(f, range(len(history.history['val_loss'])))
    np.save(f, cm)

In [ ]:
#CNN model with Batch normalization
from tensorflow.keras import regularizers
model3 = tf.keras.models.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(75, 75,
3)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.BatchNormalization(),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 #tf.keras.layers.Dense(512, activation='relu' , kernel_regularizer = regularizers.l1(0.002)),
 tf.keras.layers.Dense(1, activation='sigmoid') # 2 in case of softmax
])

In [ ]:
model3.summary()

In [ ]:
model3.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=['accuracy'])

In [ ]:
history = model3.fit(train_data,
                    train_labels, 
                    validation_data = (val_data,val_labels),
                    batch_size = 64,
                    epochs=10,
                    verbose=1)

In [ ]:
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.figure()
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
cm = show_model_result(test_data, test_labels, model3)

In [ ]:
with open('cnn_plus_BN.npy', 'wb') as f:
    np.save(f, history.history['accuracy'])
    np.save(f, history.history['val_accuracy'])
    np.save(f, history.history['loss'])
    np.save(f, history.history['val_loss'])
    np.save(f, range(len(history.history['val_loss'])))
    np.save(f, cm)

In [ ]:
#CNN Model with L1 Regularizer and Batch Normalization
from tensorflow.keras import regularizers
model4 = tf.keras.models.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(75, 75,
3)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.BatchNormalization(),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu' , kernel_regularizer = regularizers.l1(0.002)),
 tf.keras.layers.Dense(1, activation='sigmoid') # 2 in case of softmax
])

In [ ]:
model4.summary()

In [ ]:
model4.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=['accuracy'])

In [ ]:
history = model4.fit(train_data,
                    train_labels, 
                    validation_data = (val_data,val_labels),
                    batch_size = 64,
                    epochs=10,
                    verbose=1)

In [ ]:
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.figure()
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
cm = show_model_result(test_data, test_labels, model4)

In [ ]:
with open('cnn_L1_BN.npy', 'wb') as f:
    np.save(f, history.history['accuracy'])
    np.save(f, history.history['val_accuracy'])
    np.save(f, history.history['loss'])
    np.save(f, history.history['val_loss'])
    np.save(f, range(len(history.history['val_loss'])))
    np.save(f, cm)

In [ ]:
#CNN Model with Dropout layer and L2 regularizer
from tensorflow.keras import regularizers
model5 = tf.keras.models.Sequential([
 tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(75, 75,
3)),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 tf.keras.layers.BatchNormalization(),
 tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
 #tf.keras.layers.MaxPooling2D((2, 2)),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu' , kernel_regularizer = regularizers.l2(0.002)),
 tf.keras.layers.Dropout(0.2),
 tf.keras.layers.Dense(1, activation='sigmoid') # 2 in case of softmax
])

In [ ]:
model5.summary()

In [ ]:
model5.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=['accuracy'])

In [ ]:
history = model5.fit(train_data,
                    train_labels, 
                    validation_data = (val_data,val_labels),
                    batch_size = 64,
                    epochs=20,
                    verbose=1)

In [ ]:
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.figure()
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
cm = show_model_result(test_data, test_labels, model5)

In [ ]:
with open('cnn_L2_BN.npy', 'wb') as f:
    np.save(f, history.history['accuracy'])
    np.save(f, history.history['val_accuracy'])
    np.save(f, history.history['loss'])
    np.save(f, history.history['val_loss'])
    np.save(f, range(len(history.history['val_loss'])))
    np.save(f, cm)